<center>
<h1> TP-Projet d'optimisation numérique </h1>
<h1> Algorithme des régions de confiance </h1>
</center>

# Régions de confiance avec Pas de Cauchy 

## Implémentation 

1. Coder l'algorithme du pas de Cauchy dans le fichier `src/cauchy.jl`). La spécification de cet algorithme est donnée dans le fichier.
2. Ecrire des tests exhaustifs (qui testent tous les cas de figure possibles) pour votre algorithme du pas de Cauchy. Vous remplirez pour cela le fichier `test/tester_cauchy.jl` sur le modèle des autres fichiers de tests et vous exécuterez dans la cellule de code ci-après ces tests.

In [ ]:
include("../src/cauchy.jl")         # votre algorithme
include("../test/tester_cauchy.jl") # la fonction pour tester votre algorithme

#
tester_cauchy(cauchy); # tester l'algorithme

3. Coder l'algorithme des régions de confiance (fichier `src/regions_de_confiance.jl`). Sa spécification est donnée dans le fichier.
4. Vérifier que les tests ci-dessous passent.

In [11]:
include("../src/regions_de_confiance.jl")
include("../test/tester_rc_cauchy.jl")

#
afficher = true # si true, alors affiche les résultats des algorithmes

#
tester_rc_cauchy(regions_de_confiance, afficher);

TestSetException: Some tests did not pass: 12 passed, 3 failed, 0 errored, 0 broken.

## Interprétation 

<!-- Pour ces questions, des représentations graphiques sont attendues pour corroborer vos réponses. -->

1. Soit la fonction $f_1 \colon \mathbb{R}^3 \to \mathbb{R}$ définie par
$$ 
    f_1(x_1,x_2, x_3) = 2 (x_1 +x_2 + x_3 -3)^2 + (x_1-x_2)^2 + (x_2 - x_3)^2
$$ 
Quelle relation lie la fonction $f_1$ et son modèle de Taylor à l’ordre 2 ? Comparer alors les performances de l'algorithme de Newton et celui des régions de confiance avec le pas de Cauchy sur cette fonction.

2. Le rayon initial de la région de confiance est un paramètre important dans l’analyse
de la performance de l’algorithme. Sur quel(s) autre(s) paramètre(s) peut-on jouer
pour essayer d’améliorer cette performance ? Étudier l’influence d’au moins deux de
ces paramètres. Pour cela vous ferez des tests numériques et donnerez les résultats sous forme de tableaux et de graphiques.

In [ ]:
# Expérimentations numériques à faire ici
# Vous pouvez utiliser le package Plots pour les affichages de courbes: using Plots

# Régions de confiance avec gradient conjugué tronqué

## Implémentation 

1. Implémenter l’algorithme du gradient conjugué tronqué (fichier `src/gct.jl`). Sa spécification est dans le fichier.
2. Vérifier que les tests ci-dessous passent.

In [2]:
include("../src/gct.jl")
include("../test/tester_gct.jl")

#
tester_gct(gct);

Gradient conjugué tronqué: Test Failed at /home/crt4254/Documents/2A/Optimisation/TP/projet-optinum/test/tester_gct.jl:34
  Expression: ≈(s, (-Δ * g) / norm(g), atol = tol_test)
   Evaluated: [0.0, 0.0] ≈ [-0.4743416490252569, -0.15811388300841897] (atol=0.001)
Stacktrace:
 [1] macro expansion
   @ /mnt/n7fs/ens/tp_cots/julia-1.8.2/share/julia/stdlib/v1.8/Test/src/Test.jl:464 [inlined]
 [2] macro expansion
   @ ~/Documents/2A/Optimisation/TP/projet-optinum/test/tester_gct.jl:34 [inlined]
 [3] macro expansion
   @ /mnt/n7fs/ens/tp_cots/julia-1.8.2/share/julia/stdlib/v1.8/Test/src/Test.jl:1357 [inlined]
 [4] tester_gct(gct::typeof(gct))
   @ Main ~/Documents/2A/Optimisation/TP/projet-optinum/test/tester_gct.jl:23
Gradient conjugué tronqué: Test Failed at /home/crt4254/Documents/2A/Optimisation/TP/projet-optinum/test/tester_gct.jl:37
  Expression: ≈(s, [-0.8740776099190263, -0.8221850958502244], atol = tol_test)
   Evaluated: [0.0, 0.0] ≈ [-0.8740776099190263, -0.8221850958502244] (atol=0

TestSetException: Some tests did not pass: 1 passed, 8 failed, 0 errored, 0 broken.

3. Intégrer l’algorithme du gradient conjugué tronqué dans le code des régions de confiance.
4. Vérifier que les tests ci-dessous passent.

In [1]:
include("../src/regions_de_confiance.jl")
include("../test/tester_rc_gct.jl")

#
afficher = false # si true, alors affiche les résultats des algorithmes

#
tester_rc_gct(regions_de_confiance, afficher);

Affichage des résultats des algorithmes : false

RC et gct: Error During Test at /home/crt4254/Documents/2A/Optimisation/TP/projet-optinum/test/tester_rc_gct.jl:28
  Got exception outside of a @test
  MethodError: no method matching +(::Vector{Int64}, ::Int64)
  For element-wise addition, use broadcasting with dot syntax: array .+ scalar
  Closest candidates are:
    +(::Any, ::Any, ::Any, ::Any...) at operators.jl:591
    +(::T, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:87
    +(::Rational, ::Integer) at rational.jl:313
    ...
  Stacktrace:
    [1] regions_de_confiance(f::typeof(fct1), gradf::typeof(grad_fct1), hessf::typeof(hess_fct1), x0::Vector{Int64}; max_iter::Int64, tol_abs::Float64, tol_rel::Float64, epsilon::Int64, Δ0::Int64, Δmax::Int64, γ1::Float64, γ2::Int64, η1::Float64, η2::Float64, algo_pas::String, max_iter_gct::Int64)
      @ Main ~/Documents/2A/Optimisation/TP/projet-optinum/src/regions_de_confiance.jl:80

TestSetException: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.

## Interprétation  

Nous proposons de comparer l'utilisation du pas de Cauchy avec celle du gradient conjugué tronqué dans l'algorithme des régions de confiance.

**Remarques.**
* Nous vous demandons de réaliser des expérimentations numériques pour les comparaisons demandées ci-après.
* Vous devez utiliser l'argument optionnel `max_iter_gct` et la sortie `xs` de l'algorithme des régions de confiance.
* Vous pouvez comparer l'écart en norme entre les itérés de l'algorithme et la solution du problème.
* Vous trouverez des choses utiles dans le fichier `test/fonctions_de_tests.jl`.

1. Comparer dans le cas où l'on force le gradient conjugué tronqué à ne faire qu'une seule itération. Que remarquez vous ?
2. Comparer dans le cas général. Que remarquez vous ?
3. Quels sont les avantages et inconvénients des deux approches ?

In [ ]:
# Expérimentations numériques à faire ici.
# Vous pouvez utiliser le package Plots pour les affichages de courbes: using Plots